In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from tqdm.auto import tqdm

# Loading the Word Vectors

In [2]:
import pickle
from gensim.models.keyedvectors import KeyedVectors

def prepare_embedding_map(embedding_name):
    embeddings_index = {}
    if embedding_name == 'Word2Vec':
        embeddings_index = KeyedVectors.load_word2vec_format(
            '../input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin', binary=True
        )
    elif embedding_name == 'Glove300':
        embeddings_index = pickle.load(open('../input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl', 'rb'))

    else:
        if embedding_name == 'Glove50':
            f = open('../input/glove-global-vectors-for-word-representation/glove.6B.50d.txt')
        elif  embedding_name == 'Glove100':
            f = open('../input/glove-global-vectors-for-word-representation/glove.6B.100d.txt')  
        elif  embedding_name == 'Glove200':
            f = open('../input/glove-global-vectors-for-word-representation/glove.6B.200d.txt')  
        elif embedding_name == 'fastText':
            f = open('../input/fasttext-wikinews/wiki-news-300d-1M.vec')
            
        for line in tqdm(f):
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        f.close()
    return embeddings_index

In [3]:
word_to_vec_map = prepare_embedding_map('Glove300')
try:
    words = list(word_to_vec_map.vocab.keys())
except:
    words = list(word_to_vec_map.keys())
print(len(words), 'Vectors Found')    

2196008 Vectors Found


# Cosine similarity

To measure how similar two words are, we need a way to measure the degree of similarity between two embedding vectors for the two words. Given two vectors $u$ and $v$, cosine similarity is defined as follows: 

$ CosineSimilarity(u, v) = \frac {u . v} {||u||_2 ||v||_2} = cos(\theta) \tag{1}$

where $u.v$ is the dot product (or inner product) of two vectors, $||u||_2$ is the norm (or length) of the vector $u$, and $\theta$ is the angle between $u$ and $v$. This similarity depends on the angle between $u$ and $v$. If $u$ and $v$ are very similar, their cosine similarity will be close to 1; if they are dissimilar, the cosine similarity will take a smaller value. 

![](https://datascience-enthusiast.com/figures/cosine_sim.png)

In [4]:
def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similariy between u and v
        
    Arguments:
        u -- a word vector of shape (n,)          
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    
    dot = np.dot(u, v)
    norm_u = np.sqrt(np.sum(u**2))
    norm_v = np.sqrt(np.sum(v**2))
    cosine_similarity = dot / np.dot(norm_u, norm_v)
    
    return cosine_similarity

In [5]:
father = word_to_vec_map["father"]
mother = word_to_vec_map["mother"]
ball = word_to_vec_map["ball"]
crocodile = word_to_vec_map["crocodile"]
france = word_to_vec_map["france"]
italy = word_to_vec_map["italy"]
paris = word_to_vec_map["paris"]
rome = word_to_vec_map["rome"]

print("cosine_similarity(father, mother) = ", cosine_similarity(father, mother))
print("cosine_similarity(ball, crocodile) = ",cosine_similarity(ball, crocodile))
print("cosine_similarity(france - paris, rome - italy) = ",cosine_similarity(france - paris, rome - italy))

cosine_similarity(father, mother) =  0.82982457
cosine_similarity(ball, crocodile) =  0.11947029
cosine_similarity(france - paris, rome - italy) =  -0.34433752


# Word analogy

In the word analogy, we complete the sentence

<center> <b>a</b> is to <b>b</b> as <b>c</b> is to <b>__</b> </center>
    
An example is 
    
<center> <b>man</b> is to <b>woman</b> as <b>king</b> is to <b>queen</b> </center><br>

In detail, we are trying to find a word *d*, such that the associated word vectors $e_a, e_b, e_c, e_d$ are related in the following manner: $e_b - e_a \approx e_d - e_c$. We will measure the similarity between $e_b - e_a$ and $e_d - e_c$ using cosine similarity. 

In [6]:
def complete_analogy(word_a, word_b, word_c, word_to_vec_map, words):
    """
    Performs the word analogy task as explained above: a is to b as c is to ____. 
    
    Arguments:
    word_a -- a word, string
    word_b -- a word, string
    word_c -- a word, string
    word_to_vec_map -- dictionary that maps words to their corresponding vectors. 
    
    Returns:
    best_word --  the word such that v_b - v_a is close to v_best_word - v_c, as measured by cosine similarity
    """
    
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()
    
    e_a, e_b, e_c = word_to_vec_map[word_a], word_to_vec_map[word_b], word_to_vec_map[word_c]
    
    max_cosine_sim = -100
    best_word = None


    for w in words:        
        if w in [word_a, word_b, word_c] :
            continue
        
        try:
            cosine_sim = cosine_similarity((e_b - e_a), (word_to_vec_map[w] - e_c))
        except:
            pass
        
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = w
        
    return best_word

In [7]:
triads_to_try = [
    ('italy', 'italian', 'spain'), ('india', 'delhi', 'japan'),
    ('man', 'woman', 'boy'), ('small', 'smaller', 'large')
]
for triad in triads_to_try:
    print(f'{triad[0]} : {triad[1]} :: {triad[2]} : {complete_analogy(*triad, word_to_vec_map, words)}')

italy : italian :: spain : spanish
india : delhi :: japan : vihar
man : woman :: boy : girl
small : smaller :: large : larger


# Debiasing word vectors

There are some gender biases that can be reflected in a word embedding, and there are some algorithms for reducing the bias. 

Lets first see how the word embeddings relate to gender. We will first compute a vector $ g = e_{woman}-e_{man} $, where $e_{woman}$ represents the word vector corresponding to the word *woman*, and $e_{man}$ corresponds to the word vector corresponding to the word *man*. The resulting vector $g$ roughly encodes the concept of "gender".

In [8]:
g = word_to_vec_map['woman'] - word_to_vec_map['man']
print(g)

[ 1.98666990e-01  7.22199827e-02 -1.86462998e-01  5.83739996e-01
 -7.46250004e-02 -9.17997956e-03  3.19599956e-02  3.92699987e-02
  1.38819993e-01  6.78999424e-02 -2.03620002e-01  2.38368988e-01
 -2.72006691e-01 -3.08981687e-01 -1.61559999e-01 -1.46412000e-01
 -4.35483992e-01  1.89160019e-01 -2.95740008e-01 -3.50700021e-02
  9.05001163e-03  2.55119979e-01 -1.56845003e-01 -3.59719992e-01
 -2.88099945e-02  4.00590003e-01  1.07859999e-01 -2.12710008e-01
 -3.14889997e-01 -1.76756993e-01  1.34200007e-01 -5.70900142e-02
  2.26850003e-01 -2.34786004e-01  4.04179990e-01  2.45397985e-01
  1.38130009e-01 -3.17710042e-01 -6.44200146e-02  5.04499972e-02
  5.87419987e-01  1.61250010e-01 -1.27790004e-01 -5.09299934e-02
  5.82489967e-02 -1.54311001e-01 -2.74599999e-01 -4.05699909e-02
  4.65460002e-01 -2.69311011e-01 -6.96425974e-01 -7.79200047e-02
 -3.73799980e-01 -5.43542981e-01  1.68000162e-03  1.99129969e-01
  8.77784044e-02 -1.79803997e-01 -3.24763983e-01  1.82830006e-01
  4.45810035e-02 -4.40575

Now, we will consider the cosine similarity of different words with $g$

In [9]:
print ('List of names and their similarities with constructed vector:')

# girls and boys name
name_list = ['john', 'marie', 'sophie', 'ronaldo', 'priya', 'rahul', 'danielle', 'reza', 'katy', 'yasmin']

for w in name_list:
    print (w, cosine_similarity(word_to_vec_map[w], g))

List of names and their similarities with constructed vector:
john -0.1501737
marie 0.21216623
sophie 0.15576078
ronaldo -0.11417146
priya 0.097928606
rahul -0.046346735
danielle 0.12986323
reza -0.06751643
katy 0.0633719
yasmin 0.1799196


As per the above results, female first names tend to have a positive cosine similarity with our constructed vector $g$, while male first names tend to have a negative cosine similarity. This is not suprising, and the result seems acceptable. 

But let's try with some other words.

In [10]:
print('Other words and their similarities:')
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'tree', 'receptionist', 
             'technology',  'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']
for w in word_list:
    print (w, cosine_similarity(word_to_vec_map[w], g))

Other words and their similarities:
lipstick 0.25525907
guns -0.11276708
science -0.053015452
arts 0.026548538
literature 0.091985516
warrior -0.0108416155
doctor 0.1657573
tree -0.05632935
receptionist 0.2427326
technology -0.031943057
fashion 0.1786713
teacher 0.13410611
engineer -0.07418463
pilot -0.0053598527
computer 0.010542413
singer 0.10071363


It is astonishing how these results reflect certain unhealthy gender stereotypes. For example, "computer" is closer to "man" while "literature" is closer to "woman". Ouch! 

We'll see below how to reduce the bias of these vectors, using an algorithm due to [Boliukbasi et al., 2016](https://arxiv.org/abs/1607.06520). Note that some word pairs such as "actor"/"actress" or "grandmother"/"grandfather" should remain gender specific, while other words such as "receptionist" or "technology" should be neutralized, i.e. not be gender-related. We have to treat these two type of words differently when debiasing.

## Neutralize bias for non-gender specific words 

The figure below should help to visualize what neutralizing does. If you're using a n-dimensional word embedding, the n dimensional space can be split into two parts: The bias-direction $g$, and the remaining n-1 dimensions, which we'll call $g_{\perp}$. In linear algebra, we say that the n-1 dimensional $g_{\perp}$ is perpendicular (or "orthogonal") to $g$, meaning it is at 90 degrees to $g$. The neutralization step takes a vector such as $e_{receptionist}$ and zeros out the component in the direction of $g$, giving us $e_{receptionist}^{debiased}$. 

Even though $g_{\perp}$ is n-1 dimensional, given the limitations of what we can draw on a screen, we illustrate it using a 1 dimensional axis below. 

![](https://datascience-enthusiast.com/figures/neutral.png)
<caption><center>  The word vector for "receptionist" represented before and after applying the neutralize operation. </center></caption>

To remove the bias of words such as "receptionist" or "scientist". Given an input embedding $e$, we can use the following formulas to compute $e^{debiased}$: 

$$e^{bias\_component} = \frac{e \cdot g}{||g||_2^2} * g\tag{2}$$
$$e^{debiased} = e - e^{bias\_component}\tag{3}$$

$e^{bias\_component}$ is the projection of $e$ onto the direction $g$. 

<!-- 
**Reminder**: a vector $u$ can be split into two parts: its projection over a vector-axis $v_B$ and its projection over the axis orthogonal to $v$:
$$u = u_B + u_{\perp}$$
where : $u_B = $ and $ u_{\perp} = u - u_B $
!--> 

In [11]:
def neutralize(word, g, word_to_vec_map):
    """
    Removes the bias of "word" by projecting it on the space orthogonal to the bias axis. 
    This function ensures that gender neutral words are zero in the gender subspace.
    
    Arguments:
        word -- string indicating the word to debias
        g -- numpy-array of shape (n,), corresponding to the bias axis (such as gender)
        word_to_vec_map -- dictionary mapping words to their corresponding vectors.
    
    Returns:
        e_debiased -- neutralized word vector representation of the input "word"
    """
    
    e = word_to_vec_map[word]
    e_biascomponent = np.dot(e, g) / np.dot(g, g) * g
    e_debiased = e - e_biascomponent
    
    return e_debiased

In [12]:
e = "receptionist"
print(f"cosine similarity between {e} and g, before neutralizing: {cosine_similarity(word_to_vec_map[e], g)}")

e_debiased = neutralize("receptionist", g, word_to_vec_map)
print(f"cosine similarity between {e} and g, after neutralizing : {cosine_similarity(e_debiased, g)}")

cosine similarity between receptionist and g, before neutralizing: 0.24273259937763214
cosine similarity between receptionist and g, after neutralizing : -9.621787810942806e-10


## Equalization algorithm for gender-specific words

Next, lets see how debiasing can also be applied to word pairs such as "actress" and "actor." Equalization is applied to pairs of words that you might want to have differ only through the gender property. As a concrete example, suppose that "actress" is closer to "babysit" than "actor." By applying neutralizing to "babysit" we can reduce the gender-stereotype associated with babysitting. But this still does not guarantee that "actor" and "actress" are equidistant from "babysit." The equalization algorithm takes care of this. 

The key idea behind equalization is to make sure that a particular pair of words are equi-distant from the 49-dimensional $g_\perp$. The equalization step also ensures that the two equalized steps are now the same distance from $e_{receptionist}^{debiased}$, or from any other work that has been neutralized. In pictures, this is how equalization works: 

![](https://datascience-enthusiast.com/figures/equalize10.png)


The derivation of the linear algebra to do this is a bit more complex. (See Bolukbasi et al., 2016 for details.) But the key equations are: 

$$ \mu = \frac{e_{w1} + e_{w2}}{2}\tag{4}$$ 

$$ \mu_{B} = \frac {\mu \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{5}$$ 

$$\mu_{\perp} = \mu - \mu_{B} \tag{6}$$

$$ e_{w1B} = \frac {e_{w1} \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{7}$$ 
$$ e_{w2B} = \frac {e_{w2} \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{8}$$


$$e_{w1B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w1B}} - \mu_B} {||(e_{w1} - \mu_{\perp}) - \mu_B||} \tag{9}$$


$$e_{w2B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w2B}} - \mu_B} {||(e_{w2} - \mu_{\perp}) - \mu_B||} \tag{10}$$

$$e_1 = e_{w1B}^{corrected} + \mu_{\perp} \tag{11}$$
$$e_2 = e_{w2B}^{corrected} + \mu_{\perp} \tag{12}$$

In [13]:
def equalize(pair, bias_axis, word_to_vec_map):
    """
    Debias gender specific words by following the equalize method described in the figure above.
    
    Arguments:
    pair -- pair of strings of gender specific words to debias, e.g. ("actress", "actor") 
    bias_axis -- numpy-array of shape (50,), vector corresponding to the bias axis, e.g. gender
    word_to_vec_map -- dictionary mapping words to their corresponding vectors
    
    Returns
    e_1 -- word vector corresponding to the first word
    e_2 -- word vector corresponding to the second word
    """
    
    w1, w2 = pair
    e_w1, e_w2 = word_to_vec_map[w1], word_to_vec_map[w2]
    
    mu = (e_w1 + e_w2) / 2.0
    
    mu_B = np.dot(mu, bias_axis) / np.dot(bias_axis, bias_axis) * bias_axis
    mu_orth = mu - mu_B

    e_w1B = np.dot(e_w1, bias_axis) / np.dot(bias_axis, bias_axis) * bias_axis
    e_w2B = np.dot(e_w2, bias_axis) / np.dot(bias_axis, bias_axis) * bias_axis
        
    corrected_e_w1B = np.sqrt(np.abs(1 - np.dot(mu_orth, mu_orth))) * (e_w1B - mu_B) / np.linalg.norm(e_w1 - mu_orth - mu_B)
    corrected_e_w2B = np.sqrt(np.abs(1 - np.dot(mu_orth, mu_orth))) * (e_w2B - mu_B) / np.linalg.norm(e_w2 - mu_orth - mu_B)

    e1 = corrected_e_w1B + mu_orth
    e2 = corrected_e_w2B + mu_orth
                                                                
    return e1, e2

In [14]:
print("cosine similarities before equalizing:")
print("cosine_similarity(word_to_vec_map[\"man\"], gender) = ", cosine_similarity(word_to_vec_map["man"], g))
print("cosine_similarity(word_to_vec_map[\"woman\"], gender) = ", cosine_similarity(word_to_vec_map["woman"], g))
print()
e1, e2 = equalize(("man", "woman"), g, word_to_vec_map)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, g))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, g))

cosine similarities before equalizing:
cosine_similarity(word_to_vec_map["man"], gender) =  -0.2595359
cosine_similarity(word_to_vec_map["woman"], gender) =  0.45727175

cosine similarities after equalizing:
cosine_similarity(e1, gender) =  -0.702331
cosine_similarity(e2, gender) =  0.70233107


# Reference

[Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings](https://proceedings.neurips.cc/paper/2016/file/a486cd07e4ac3d270571622f4f316ec5-Paper.pdf)